In [26]:
from utils import * 
from Fed import * 

In [27]:


class MyMIA : 

    def __init__(self, target_data, similar_data, target_model, target_model_fn, attack_model_fn, params) : 
        self.target_data = target_data
        self.similar_data = similar_data
        self.target_model = target_model
        self.target_model_fn = target_model_fn
        self.attack_model_fn = attack_model_fn
        self.params = params

        # We assume that attacker's data were not seen in target's training.
        self.similar_x_in, self.similar_x_out, self.similar_y_in, self.similar_y_out = train_test_split(
            similar_data[0], similar_data[1], test_size=0.1
        )


    def train_shadows(self) :
        # Train the shadow models.
        
        self.smb = ShadowModelBundle(
            self.target_model_fn, 
            shadow_dataset_size=len(self.similar_x_in) // self.params['num_shadows'],
            num_models=self.params['num_shadows']
        )
        
        print("Training the shadow models...")
        self.x_attack, self.y_attack = self.smb.fit_transform(
            self.similar_x_in,
            self.similar_y_in,
            fit_kwargs=dict(
                epochs=self.params['shadow_epochs'],
                verbose=False,
                validation_data=(self.similar_x_out, self.similar_y_out),
            ),
        )


    def train_attacker(self) :
        # ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
        
        self.amb = AttackModelBundle(self.attack_model_fn, num_classes= self.params['num_classes'])

        # Fit the attack models.
        print("Training the attack models...")
        self.amb.fit(
            self.x_attack, self.y_attack
        )


    def test_attack(self) : 
        # Prepare examples that were in the training, and out of the training.
        data_in = self.target_data
        data_out = self.similar_data

        # Compile them into the expected format for the AttackModelBundle.
        attack_test_data, real_membership_labels = prepare_attack_data(
            self.target_model, data_in, data_out
        )

        attack_guesses = self.amb.predict(attack_test_data)
        attack_tp = np.mean((attack_guesses == 1) == (real_membership_labels == 1))
        self.attack_accuracy = np.mean(attack_guesses == real_membership_labels)


        return self.attack_accuracy, classification_report(real_membership_labels, attack_guesses)



## FEDERATED LEARNING

In [28]:
args = argparse.Namespace(
    dataset='cifar10',
    learning_algorithm='fedakd',
    num_clients=5,
    local_size=5000,
    rounds=20,
    local_epochs=2,
    batch_size=32,
    proxy_data_size=5000,
    early_stop_patience=10,
    lr = 0.001,
    lr_reduction_patience=10,
    target_model='nn',
    dp_epsilon=0.5,
    dp_delta=1e-5,
    use_dp=True,
    dp_type = 'dp',
    dp_norm_clip=1.0,
    temperature=0.7,
    aalpha=1000,
    bbeta=1000,
)

# iterate over the parameters of args 
params = dict()
for k, v in vars(args).items():
    params[k] = v

In [3]:




experiment_id = args.dataset + '_' + args.learning_algorithm + '_' + datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
experiment_dir = os.path.join(RESULTS_PATH, experiment_id)
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)



train_data, test_data, metadata = get_data(args.dataset)
args.input_shape = metadata['input_shape']
args.num_classes = metadata['num_classes']
args.class_labels = metadata['class_labels']

train_data = (np.array(train_data[0] / 255, dtype=np.float32), tf.keras.utils.to_categorical(train_data[1]))
test_data = (np.array(test_data[0] / 255, dtype=np.float32), tf.keras.utils.to_categorical(test_data[1]))



centralized_data, clients_data, external_data = split_data(train_data, args.num_clients, args.local_size)

proxy_limit = args.proxy_data_size
proxy_data, _, _ = get_data('cifar100')
proxy_data = (np.array(proxy_data[0][:proxy_limit] / 255, dtype=np.float32), tf.keras.utils.to_categorical(proxy_data[1][:proxy_limit]))

# exp_path, clients_data, test_data, initial_model, args
# learning_algorithm = FedAKD(experiment_dir, clients_data, test_data, proxy_data, create_model_based_on_data, args)
initial_model = create_model_based_on_data(args, compile_model = False)
learning_algorithm = FedAvg(experiment_dir, clients_data, test_data, initial_model, args)
# t_model = learning_algorithm.create_temperature_scaled_model(initial_model, args.temperature)




Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-23 19:31:48.418062: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-23 19:31:48.418508: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
def new_aggregate(weights) : 
    weight_accumulator = None
    for weight in weights :
        if weight_accumulator is None:
            # If accumulator is uninitialized, set it to the client weights
            weight_accumulator = weight
        else:
            # Add the client weights to the accumulator
            weight_accumulator = [
                accumulator + client_weight
                for accumulator, client_weight in zip(weight_accumulator, weights)
        ]
    weight_accumulator = [accumulator / len(clients_models) for accumulator in weight_accumulator]
    return weight_accumulator

In [18]:
weights = [model.get_weights() for model in learning_algorithm.clients_models]


# new_weights = np.empty_like(weights[0]) 

# for i in range(len(weights[0])) :
#     for j in range(len(weights[0][i])) :
#         new_weights[i][j] = np.sum([weight[i][j] for weight in weights])


In [25]:

avg_weights =[] 
for layer_id in range(len(weights[0]) ): 
    avg_layer = np.mean([weights[i][layer_id] for i in range(len(weights))], axis = 0)
    avg_weights.append(avg_layer)

learning_algorithm.clients_models[0].set_weights(avg_weights)

## Central MIA


In [ ]:

eepochs = np.arange(1, 50, 5)
accs = [] 
for eep in eepochs :

    args = argparse.Namespace(
        dataset='cifar10',
        learning_algorithm='central',
        num_clients=5,
        local_size=500,
        rounds=30,
        local_epochs=eep,
        proxy_data_size=1000,
        early_stop_patience=10,
        lr_reduction_patience=10,
        target_model='nn',
        epsilon=0.5,
        dp='dp'
    )

    experiment_id = args.dataset + '_' + args.learning_algorithm + '_' + datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
    experiment_dir = os.path.join(RESULTS_PATH, experiment_id)
    if not os.path.exists(experiment_dir):
        os.makedirs(experiment_dir)


    train_data, test_data, metadata = get_data(args.dataset)

    train_data = (np.array(train_data[0] / 255, dtype=np.float32), tf.keras.utils.to_categorical(train_data[1]))
    test_data = (np.array(test_data[0] / 255, dtype=np.float32), tf.keras.utils.to_categorical(test_data[1]))


    centralized_model = create_model_based_on_data(metadata)
    callbacks = {
        'early_stop_patience' : args.early_stop_patience,
        'lr_reduction_patience' : args.lr_reduction_patience,
        'csv_logger_path' : join(experiment_dir, 'centralized.csv')
    }
    history = train_keras_model(centralized_model, train_data, test_data, epochs=args.local_epochs, verbose=0, **callbacks)



    mia_params = {
        'num_shadows': 3,  # 3 shadow models
        'num_classes': 10, # number of output classes
        'shadow_epochs': 10,
        'attack_epochs': 10,
        
    }

    mia = MyMIA(train_data, test_data, centralized_model, create_model_based_on_data, attack_model_fn, mia_params)

    mia.train_shadows() 

    mia.train_attacker()

    acc, clf_rep = mia.test_attack()
    print("epoch : ", eep, " acc : ", acc)
    accs.append(acc)


In [ ]:
plt.plot(accs)